<a href="https://colab.research.google.com/github/haeunY153/DL-Practice/blob/main/KoBERT_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Masked Language Model
1) 입력 텍스트의 단어집합의 15%를 랜덤으로 마스킹
- `마스킹`: 가려서 학습이 안되게 하는 기법

2) 인공신경망을 통해 마스킹 된 단어들을 예측
- 문장 중간에 구멍을 뚫어놓고, 구멍에 들어갈 단어들을 예측

### 사전 학습된 한국어 BERT를 이용하여 마스크드 언어 모델 실습
- `transformers`라는 패키지를 주로 사용

In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 6.2 MB/s 
     |████████████████████████████████| 7.6 MB 47.6 MB/s 
     |████████████████████████████████| 163 kB 56.4 MB/s 


**BERT는 이미 누군가가 학습해둔 모델을 사용하는 것이므로 우리가 사용하는 모델과 토크나이저는 항상 맵핑 관계여야함**
- `klue/bert-base`는 대표적인 한국어 BERT

In [2]:
from transformers import TFBertForMaskedLM #마스크드 언어 모델링 모듈
from transformers import AutoTokenizer

#AutoTokenizer.from_pretrained('모델 이름'): 학습되었던 토크나이저 출력

model = TFBertForMaskedLM.from_pretrained('klue/bert-base', from_pt=True) #from_pt: 텐서플로우 활용
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForMaskedLM: ['cls.predictions.decoder.bias', 'bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForMaskedLM from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForMaskedLM from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

#### 1. 임의의 문장 BERT 입력하기

In [3]:
inputs = tokenizer('축구는 정말 재미있는 [MASK]다.', return_tensors='tf')
print(inputs['input_ids']) # 정수 인코딩 결과 확인

tf.Tensor([[   2 4713 2259 3944 6001 2259    4  809   18    3]], shape=(1, 10), dtype=int32)


In [4]:
print(inputs['token_type_ids']) # 문장을 구분하는 세그먼트 구할수 있음 (예시가 문장1개여서 구분값 없음)

tf.Tensor([[0 0 0 0 0 0 0 0 0 0]], shape=(1, 10), dtype=int32)


In [5]:
print(inputs['attention_mask'])

tf.Tensor([[1 1 1 1 1 1 1 1 1 1]], shape=(1, 10), dtype=int32)


#### 2. [MASK] 토큰 예측하기

In [6]:
from transformers import FillMaskPipeline
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)

In [9]:
# [MASK]에 들어갈 수 있는 상위 5개 단어후보 반환

pip('축구는 정말 재미있는[MASK]다.')

[{'score': 0.8963514566421509,
  'token': 4559,
  'token_str': '스포츠',
  'sequence': '축구는 정말 재미있는 스포츠 다.'},
 {'score': 0.02595745027065277,
  'token': 568,
  'token_str': '거',
  'sequence': '축구는 정말 재미있는 거 다.'},
 {'score': 0.010033912025392056,
  'token': 3682,
  'token_str': '경기',
  'sequence': '축구는 정말 재미있는 경기 다.'},
 {'score': 0.007924332283437252,
  'token': 4713,
  'token_str': '축구',
  'sequence': '축구는 정말 재미있는 축구 다.'},
 {'score': 0.007844172418117523,
  'token': 5845,
  'token_str': '놀이',
  'sequence': '축구는 정말 재미있는 놀이 다.'}]

In [11]:
pip('나는 내일 [MASK]로 놀러 간다.')

[{'score': 0.05731149762868881,
  'token': 1,
  'token_str': '[UNK]',
  'sequence': '나는 내일 로 놀러 간다.'},
 {'score': 0.035766489803791046,
  'token': 9937,
  'token_str': 'LA',
  'sequence': '나는 내일 LA 로 놀러 간다.'},
 {'score': 0.024779921397566795,
  'token': 16,
  'token_str': ',',
  'sequence': '나는 내일, 로 놀러 간다.'},
 {'score': 0.019969750195741653,
  'token': 3671,
  'token_str': '서울',
  'sequence': '나는 내일 서울 로 놀러 간다.'},
 {'score': 0.013639144599437714,
  'token': 7095,
  'token_str': '도쿄',
  'sequence': '나는 내일 도쿄 로 놀러 간다.'}]

In [12]:
pip('[MASK]를 먹고 기분이 너무 좋았다.') # 결과 예측이 너무 안 됨

[{'score': 0.07000207901000977,
  'token': 34,
  'token_str': '>',
  'sequence': '> 를 먹고 기분이 너무 좋았다.'},
 {'score': 0.0637006089091301,
  'token': 1,
  'token_str': '[UNK]',
  'sequence': '를 먹고 기분이 너무 좋았다.'},
 {'score': 0.02703658491373062,
  'token': 13,
  'token_str': ')',
  'sequence': ') 를 먹고 기분이 너무 좋았다.'},
 {'score': 0.02658284455537796,
  'token': 17,
  'token_str': '-',
  'sequence': '- 를 먹고 기분이 너무 좋았다.'},
 {'score': 0.020930781960487366,
  'token': 7,
  'token_str': '#',
  'sequence': '# 를 먹고 기분이 너무 좋았다.'}]

In [13]:
inputs = tokenizer('[MASK]를 먹고 기분이 너무 좋았다.', return_tensors='tf')

In [16]:
print(inputs['input_ids']) 

tf.Tensor([[   2    4 1022 1059 2088 4754 2052 3760 1560 2886 2062   18    3]], shape=(1, 13), dtype=int32)


In [17]:
# 학습하고 다시 시도

pip('[MASK]를 먹고 기분이 너무 좋았다.')

[{'score': 0.07000207901000977,
  'token': 34,
  'token_str': '>',
  'sequence': '> 를 먹고 기분이 너무 좋았다.'},
 {'score': 0.0637006089091301,
  'token': 1,
  'token_str': '[UNK]',
  'sequence': '를 먹고 기분이 너무 좋았다.'},
 {'score': 0.02703658491373062,
  'token': 13,
  'token_str': ')',
  'sequence': ') 를 먹고 기분이 너무 좋았다.'},
 {'score': 0.02658284455537796,
  'token': 17,
  'token_str': '-',
  'sequence': '- 를 먹고 기분이 너무 좋았다.'},
 {'score': 0.020930781960487366,
  'token': 7,
  'token_str': '#',
  'sequence': '# 를 먹고 기분이 너무 좋았다.'}]

In [18]:
pip('자기 전에 꼭 [MASK]하는 습관이 있다.')

[{'score': 0.39163729548454285,
  'token': 17313,
  'token_str': '스트레칭',
  'sequence': '자기 전에 꼭 스트레칭 하는 습관이 있다.'},
 {'score': 0.1451481282711029,
  'token': 3645,
  'token_str': '해야',
  'sequence': '자기 전에 꼭 해야 하는 습관이 있다.'},
 {'score': 0.060628216713666916,
  'token': 11030,
  'token_str': '지켜야',
  'sequence': '자기 전에 꼭 지켜야 하는 습관이 있다.'},
 {'score': 0.02739705517888069,
  'token': 22327,
  'token_str': '챙겨야',
  'sequence': '자기 전에 꼭 챙겨야 하는 습관이 있다.'},
 {'score': 0.02706955559551716,
  'token': 1,
  'token_str': '[UNK]',
  'sequence': '자기 전에 꼭 하는 습관이 있다.'}]

- 문장에서 마스크의 위치에 따라 예측을 자연스럽게 하기도, 못하기도 하는 것 같다.